In [1]:
import os
import time

import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

import sys
sys.path.append('../python/administrativeArea/')
from vietnamADM import AdministrativeArea

In [2]:
cluster = LocalCluster(ip="0.0.0.0")
client = Client(cluster)
client

Client Scheduler: tcp://10.3.0.79:46027 Dashboard: http://10.3.0.79:8787/status,Cluster Workers: 8 Cores: 40 Memory: 270.38 GB


In [3]:
N_JOBS=32

In [4]:
from_date = '2018-11-14'
end_date = '2018-11-27'
PATH = '/home/phongdk/data_user_income_targeting'
VNM_ADM_PATH = '/home/phongdk/VNM_adm/'
filename_location = "location_from_{}_to_{}.csv.gz".format(from_date, end_date)

vn_adm = AdministrativeArea(VNM_ADM_PATH)
# vn_adm2 = AdministrativeArea2(os.path.join(VNM_ADM_PATH, 'VNM_adm2.shp'))
# vn_adm3 = AdministrativeArea2(os.path.join(VNM_ADM_PATH, 'VNM_adm3.shp'))


In [5]:
%%time
df = pd.read_csv(os.path.join(PATH, filename_location), dtype={'user_id': str})#, nrows=5000)
# print('--------Finding address----------------')
# df.apply(lambda x: vn_adm.find_address(x['lat'], x['lon']), axis=1)
# CPU times: user 22h 45min 27s, sys: 24min 11s, total: 23h 9min 39s
# Wall time: 22h 38min 29s    

CPU times: user 27.7 s, sys: 2.36 s, total: 30 s
Wall time: 28 s


In [6]:
%%time
df_dask = dd.from_pandas(df, npartitions=N_JOBS)
#df_dask['address'] = df_dask.apply(lambda x: vn_adm.find_address(x['lat'], x['lon']), meta={'lat': float, 'lon':float, 'address':str}, axis=1).compute()
address = df_dask.apply(lambda x: vn_adm.find_address(x['lat'], x['lon']), axis=1).compute()
df_dask = df_dask.assign(**{'address': address})

/home/phongdk/anaconda3/lib/python3.6/site-packages/dask/dataframe/core.py:3107: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)
/home/phongdk/anaconda3/lib/python3.6/site-packages/distributed/worker.py:2791: UserWarning: Large object of size 72.46 MB detected in task graph: 
  (                       user_id       lat      lon ... 97abaf451fc6f')
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s))


CPU times: user 22min 15s, sys: 5min 20s, total: 27min 35s
Wall time: 3h 24min 55s


In [13]:
df_dask = dd.from_pandas(df_dask[['user_id', 'address']].drop_duplicates(subset=['user_id', 'address']).compute(), npartitions=N_JOBS)  #pandas
df_dask.to_csv(os.path.join(PATH, "full_location/address_from_{}_to_{}_file*.csv".format(from_date, end_date)), index=False)

['/home/phongdk/data_user_income_targeting/full_location/address_from_2018-11-14_to_2018-11-27_file00.csv',
 '/home/phongdk/data_user_income_targeting/full_location/address_from_2018-11-14_to_2018-11-27_file01.csv',
 '/home/phongdk/data_user_income_targeting/full_location/address_from_2018-11-14_to_2018-11-27_file02.csv',
 '/home/phongdk/data_user_income_targeting/full_location/address_from_2018-11-14_to_2018-11-27_file03.csv',
 '/home/phongdk/data_user_income_targeting/full_location/address_from_2018-11-14_to_2018-11-27_file04.csv',
 '/home/phongdk/data_user_income_targeting/full_location/address_from_2018-11-14_to_2018-11-27_file05.csv',
 '/home/phongdk/data_user_income_targeting/full_location/address_from_2018-11-14_to_2018-11-27_file06.csv',
 '/home/phongdk/data_user_income_targeting/full_location/address_from_2018-11-14_to_2018-11-27_file07.csv',
 '/home/phongdk/data_user_income_targeting/full_location/address_from_2018-11-14_to_2018-11-27_file08.csv',
 '/home/phongdk/data_user_in

In [8]:
%%time
provinces = df_dask['address'].apply(lambda x: x.split(',')[-1].strip()).compute()
df_dask = df_dask.assign(**{'province': provinces})
df_user_province= df_dask[['user_id', 'province']].drop_duplicates(subset=['user_id', 'province']).compute()  #pandas
print(df_user_province.head())
print('Length after dropping duplicates', len(df_user_province))
df_dask_user_province = dd.from_pandas(df_user_province, npartitions=N_JOBS)
user_list_provinces = df_dask_user_province.groupby('user_id')['province'].apply(list).compute()  # pandas
print(user_list_provinces.head())
print('Length after grouping provinces', len(df_user_province))
df_dask_user_list_provinces = dd.from_pandas(user_list_provinces, npartitions=N_JOBS)

/home/phongdk/anaconda3/lib/python3.6/site-packages/dask/dataframe/core.py:2284: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


           user_id          province
0    4179731125044        Bình Thuận
1   10846855546626  Hồ Chí Minh city
6   15415225686218        Bình Dương
10  16957361450243           Cần Thơ
11  19187710618804            Hà Nội
Length after dropping duplicates 9735668


/home/phongdk/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  import sys


user_id
10000087237517407770    [Hồ Chí Minh city, Đắk Lắk]
10000102880679195398                       [Hà Nội]
10000124325894525605                       [Hà Nội]
10000182051201904318            [Nghệ An, Lạng Sơn]
10000256181516297049                       [Hà Nội]
Name: province, dtype: object
Length after grouping provinces 9735668
CPU times: user 1min 51s, sys: 20.4 s, total: 2min 11s
Wall time: 5min 42s


In [9]:
#%%time
#user_list_province = df_dask_user_province.groupby('user_id')['province'].apply(list).compute()

In [14]:
df_dask_user_list_provinces.to_csv(os.path.join(PATH, "address_from_{}_to_{}_file*.csv".format(from_date, end_date)), index=True)

['/home/phongdk/data_user_income_targeting/address_from_2018-11-14_to_2018-11-27_file00.csv',
 '/home/phongdk/data_user_income_targeting/address_from_2018-11-14_to_2018-11-27_file01.csv',
 '/home/phongdk/data_user_income_targeting/address_from_2018-11-14_to_2018-11-27_file02.csv',
 '/home/phongdk/data_user_income_targeting/address_from_2018-11-14_to_2018-11-27_file03.csv',
 '/home/phongdk/data_user_income_targeting/address_from_2018-11-14_to_2018-11-27_file04.csv',
 '/home/phongdk/data_user_income_targeting/address_from_2018-11-14_to_2018-11-27_file05.csv',
 '/home/phongdk/data_user_income_targeting/address_from_2018-11-14_to_2018-11-27_file06.csv',
 '/home/phongdk/data_user_income_targeting/address_from_2018-11-14_to_2018-11-27_file07.csv',
 '/home/phongdk/data_user_income_targeting/address_from_2018-11-14_to_2018-11-27_file08.csv',
 '/home/phongdk/data_user_income_targeting/address_from_2018-11-14_to_2018-11-27_file09.csv',
 '/home/phongdk/data_user_income_targeting/address_from_2018